In this notebook, we will learn to implement a very simple SMPC model to reproduce the toy example of the beers

In [ ]:
import random


Let's define the encryption function.

First, we assume to have a finite field to which the secrets and the shares belong. To do that, we define a very large prime number `Q` that acts as a modulus.

Then, we can split a secret into `n_shares` shares by simply extracting `n_shares - 1` random numbers from this field and computing the last share accordingly.

In [ ]:
Q = 121639451781281043402593

def encrypt(x, n_shares=2):
    shares = list()
    for i in range(n_shares - 1):
        shares.append(random.randint(0, Q))
    final_share = x - (sum(shares) % Q)
    shares.append(final_share)
    return tuple(shares)

print(encrypt(100, 3))


The secret can be then reconstructed by summing up the secrets.

In [ ]:
def decrypt(shares):
    return sum(shares) % Q

Let's consider now the example of the beers.

Three friends have a secret about the number of beers they drank. Let's put their secrets in a list.

In [ ]:
secrets = [100, 200, 300]

Now, we can encrypt their secrets into three shares each and put them into `local_shares`.

In [ ]:
local_shares = [[], [], []]

for s in secrets:
    t = encrypt(s, 3)
    for i in range(len(t)):
        local_shares[i].append(t[i])

print('Local shares of the secrets:', local_shares)

Each party has now a piece of information about itself and the other two parties. The scheme we built has some homomorphic properties:
* we can sum shares (e.g., if `x_1`, `x_2` is a sharing of `x` and `y_1`, `y_2` is a sharing of `y`, then `x_1 + y_1`, `x_2 + y_2` is a sharing of `x + y`);
* we can sum or multiply shares with non-private values.

In [ ]:
local_computation = [sum(u) % Q for u in local_shares]

Now, each party has a share of the sum of the secrets. We can now decrypt the final secret (the sum of the initial secrets) with the function created above.

In [ ]:
print('Local shares of the sum of the secrets:', local_computation)
print(decrypt(local_computation) / len(local_computation))

Second part:

In [ ]:
# We now integrate the SPDZ protocol, which assumes the presence of a cryptoprovider

def generate_mul_triple():
    a = random.randrange(Q)
    b = random.randrange(Q)
    a_mul_b = (a * b) % Q
    return encrypt(a), encrypt(b), encrypt(a_mul_b)
